Google Earth Engine Python API code to convert a multiband image to an image collection.

This code specifically shows how to take precipitation data released by [India Meteorological Department](http://imdpune.gov.in/Clim_Pred_LRF_New/Grided_Data_Download.html) and convert it into a form that is usable in Earth Engine. 

Another notebook in this repository `imd_to_geotiff.ipynb` takes the gridded binary data and converts it to a GeoTiff file. Each geotiff file has 1 band per day for a given year. Once uploaded to Earth Engine, this notebook shows, how to take each band and write it to its separate asset, resulting in an ImageCollection object which is the preferred container to work with time series data such as this.

In [40]:
import ee
ee.Authenticate()

Enter verification code:  4/5QHI0Z6zxnX8KteDjKolJUZEkk5T1Pl3Yrca5qe8T1d6TPJevln2cIc



Successfully saved authorization token.


In [41]:
ee.Initialize()

Function to take a year image and export it

In [42]:
def export_year_image(year):
    imageId = 'users/ujavalgandhi/external/imd/{}'.format(year)
    outputCollectionId =  'users/ujavalgandhi/imd/rainfall'
    image = ee.Image(imageId)
    daysList = ee.List.sequence(0, image.bandNames().length().subtract(1))

    def bandToDay(day):
        dayBand = image.select([day]).rename('rainfall')
        date = ee.Date.fromYMD(year, 1, 1).advance(day, 'day')
        return dayBand.set('system:time_start', date.millis())

    dailyImages = daysList.map(bandToDay)

    dailyCol = ee.ImageCollection.fromImages(dailyImages)
    image_ids = dailyCol.aggregate_array('system:index').getInfo()
    print('Year {}, Total images: {}'.format(year, len(image_ids)))
    
    for i, image_id in enumerate(image_ids):
        exportImage = ee.Image(dailyCol.filter(ee.Filter.eq('system:index', image_id)).first())
        exportAssetId = '{}/{}_{}'.format(outputCollectionId, year, i+1)
        task = ee.batch.Export.image.toAsset(**{
            'image': exportImage,
            'description': 'Image to Asset {}'.format(i+1),
            'assetId': exportAssetId,
            'scale': exportImage.projection().nominalScale().getInfo(),
            'region': exportImage.geometry().getInfo()['coordinates'],
            'maxPixels': 1e10
        })
        task.start()
        print('Started Task: ', i+1)

Export all the images to a single collection

In [ ]:
years = [2014, 2015, 2016]
for year in years:
    export_year_image(year)

Year 2013, Total images: 365
Started Task:  1
Started Task:  2
Started Task:  3
Started Task:  4
Started Task:  5
Started Task:  6
Started Task:  7
Started Task:  8
Started Task:  9
Started Task:  10
Started Task:  11
Started Task:  12
Started Task:  13
Started Task:  14
Started Task:  15
Started Task:  16
Started Task:  17
Started Task:  18
Started Task:  19
Started Task:  20
Started Task:  21
Started Task:  22
Started Task:  23
Started Task:  24
Started Task:  25
Started Task:  26
Started Task:  27
Started Task:  28
Started Task:  29
Started Task:  30
Started Task:  31
Started Task:  32
Started Task:  33
Started Task:  34
Started Task:  35
Started Task:  36
Started Task:  37
Started Task:  38
Started Task:  39
Started Task:  40
Started Task:  41
Started Task:  42
Started Task:  43
Started Task:  44
Started Task:  45
Started Task:  46
Started Task:  47
Started Task:  48
Started Task:  49
Started Task:  50
Started Task:  51
Started Task:  52
Started Task:  53
Started Task:  54
Started 